<a href="https://colab.research.google.com/github/ARAN1218/RealEstateRentPrediction_AI/blob/main/jupyter%20notebook/%E4%B8%8D%E5%8B%95%E7%94%A3%E8%B3%83%E6%96%99%E4%BA%88%E6%B8%ACAI(SUUMO).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
from time import sleep
from pprint import pprint
import json

import numpy as np
import pandas as pd
pd.set_option('display.max_rows',None)
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import xgboost as xgb

In [ ]:
#endは偶数
start = 1
end = 20
place = '相模原'

urls = {
    '相模原':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=14&sc=14151&sc=14152&sc=14153&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}",
    '渋谷':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=13&sc=13113&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}",
    '新宿':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=13&sc=13104&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}"
}
#urls[place]
for i in range(start,end+1,2):
    print(i)
    
for i in range(start+1,end+1,2):
    print(i)

In [ ]:
#相模原orの物件の賃料データスクレイピングの関数
#Real_Estate_Data_Scraping
def REDS(start,end,place):
    
    #初期値
    d_list = []
    url = urls[place]
    
    for i in range(start,end+1,2):
    
        #ページを遷移させる
        target_url = url.format(i)

        #Requestsを用いてtarget_urlにアクセスする
        r = requests.get(target_url)

        #サーバー負荷軽減の為、ループ毎に1秒間隔を空ける
        sleep(1)

        #取得したHTMLをBeautifulSoupで解析する
        soup = BeautifulSoup(r.text)

        contents = soup.find_all('div',class_='cassettebox js-normalLink js-cassetLink')
        for content in contents:

            rows = content.find_all('table',class_='listtable')
            #title = content.find('a',class_='js-cassetLinkHref').text.split()[1]
            address = rows[0].find_all('div',class_='infodatabox-box-txt')[0].text
            station = rows[0].find_all('div',class_='infodatabox-box-txt')[1].text
            access = rows[0].find_all('div',class_='infodatabox-box-txt')[2].text

            #下の行のデータを取得
            r_fees = rows[1].find_all('dd',class_='infodatabox-details-txt')[0].text[:-2]
            mc_fees = rows[1].find_all('dd',class_='infodatabox-details-txt')[1].text[:-1]
            k_fees = rows[1].find_all('dd',class_='infodatabox-details-txt')[2].text.split('/')[0]
            s_fees = rows[1].find_all('dd',class_='infodatabox-details-txt')[2].text.split('/')[1][:-2]
            area = rows[1].find_all('dd',class_='infodatabox-details-txt')[3].text[:-2]
            layout = rows[1].find_all('dd',class_='infodatabox-details-txt')[4].text
            age = rows[1].find_all('div',class_='infodatabox-box-txt')[2].text

            #取得した各種データを辞書dに格納する
            d = {
            '住所':address,
            '路線':station,
            '交通':access,
            '賃料':r_fees,
            '管理共益費':mc_fees,
            '礼金':k_fees,
            '敷金':s_fees,
            '専有面積':area,
            '間取り':layout,
            '築年数':age
            }

            #辞書dのデータをリストd_listに格納する
            d_list.append(d)
            
            #重複したデータを削除する
            d_list = list(map(json.loads,set(map(json.dumps,d_list))))

        #進捗を報告させる
        print("d_list's progress:",i,"page　　",len(d_list))
        print(target_url)
        
        #データ数が100以上でない場合、スクレイピングを延長する
        #if(((i==end-1) or (i==end)) and (len(d_list) < 100)):
            #print('extended!')
            #end+=2
    
    print('Scraping Completed!')
    return d_list


In [ ]:
d_list = REDS(start,end,place)

In [ ]:
df = pd.DataFrame(d_list)
df.head(10)

In [ ]:
d_t_list = REDS(start+1,end+1,place)

In [ ]:
df_test = pd.DataFrame(d_t_list)
df_test.head(10)

In [ ]:
pprint(d_list)

In [ ]:
#dfの異なるデータの数を確認する
#print(len(df.住所.unique()))
#print(len(df.専有面積.unique()))
#print(len(df.交通.unique()))
print(df.duplicated().sum())
print(len(df))

In [ ]:
#重複した行を削除してみる
df = df.drop_duplicates()
df.head(300)

In [ ]:
#df,df_test内の余計な文字(\n等)を消去する
df['礼金'] = df['礼金'].str.replace(r'\(.+\)','')
for element in ['住所','交通','専有面積','間取り','築年数','礼金','敷金']:
    df[element] = df[element].str.translate(str.maketrans({'-':'','\r':'','\n':'','\t':'','バ':'','ス':'','分':'','徒':'','歩':'','停':'','―':'','築':'','年':'','新':'','万':'','(':'','円':''}))


df_test['礼金'] = df_test['礼金'].str.replace(r'\(.+\)','')
for element in ['住所','交通','専有面積','間取り','築年数','礼金','敷金']:
    df_test[element] = df_test[element].str.translate(str.maketrans({'-':'','\r':'','\n':'','\t':'','バ':'','ス':'','分':'','徒':'','歩':'','停':'','―':'','築':'','年':'','新':'','万':'','(':'','円':''}))
df.head(100)

In [ ]:
'''
#df内の余計な文字(\n等)を消去する
df['住所'] = df['住所'].str.translate(str.maketrans({'\r':'','\n':'','\t':''}))
df['交通'] = df['交通'].str.translate(str.maketrans({'バ':'','ス':'','分':'','徒':'','歩':'','停':'','―':''}))
df['専有面積'] = df['専有面積'].str.translate(str.maketrans({'\r':'','\n':'','\t':''}))
df['間取り'] = df['間取り'].str.translate(str.maketrans({'\r':'','\n':'','\t':''}))
df['築年数'] = df['築年数'].str.translate(str.maketrans({'\r':'','\n':'','\t':'','築':'','年':'','新':''}))
df['礼金'] = df['礼金'].str.replace(r'\(.+\)','')
df['礼金'] = df['礼金'].str.translate(str.maketrans({'-':'','万':'','(':'','円':''}))
df['敷金'] = df['敷金'].str.translate(str.maketrans({'-':'','万':'','(':''}))
df.head(10)
'''

In [ ]:
#ラベルエンコーディングを行い、文字列を数値化する
LE1,LE2,LE3 = LabelEncoder(),LabelEncoder(),LabelEncoder()
#for i in ['住所','路線','間取り']:
    #df[i] = LE.fit_transform(df[i])
LE_df = pd.concat([df,df_test])
LE1.fit(LE_df['住所'])
LE2.fit(LE_df['路線'])
LE3.fit(LE_df['間取り'])

df['住所'] = LE1.transform(df['住所'])
df['路線'] = LE2.transform(df['路線'])
df['間取り'] = LE3.transform(df['間取り'])
df.head(10)

In [ ]:
adressC,stationC,layoutC = LE1.classes_,LE2.classes_,LE3.classes_
print(adressC,'\n',stationC,'\n',layoutC)

In [ ]:
#欠損値をどうにかする
df.replace('',0,inplace=True)
df.head(10)

In [ ]:
df.head(len(df))

In [ ]:
#データ型をfloat64で統一する。
df = df.astype('float64')
#df['礼金'] = df['礼金'].astype('float64')
print(df.dtypes)
df.head(10)

In [ ]:
#不動産賃料予測AIの学習
#Real_Estate_Rent_Learning
def RERL(df):
    #データを説明変数と目的変数に分ける
    target = df['賃料']
    train = df.drop(['賃料'],axis=1)
    
    #ホールドアウト法で学習データとテストデータに分ける
    kf = KFold(n_splits=4,shuffle=True,random_state=71)
    tr_idx,va_idx = list(kf.split(train))[0]
    tr_x,va_x = train.iloc[tr_idx],train.iloc[va_idx]
    tr_y,va_y = target.iloc[tr_idx],target.iloc[va_idx]
    
    #データ型をXGBOOST用に適合させる
    dtrain = xgb.DMatrix(tr_x,label=tr_y)
    dvalid = xgb.DMatrix(va_x,label=va_y)
    dtest = xgb.DMatrix(train)
    
    #パラメータ準備
    params = {'objective':'reg:squarederror','silent':1,'random_state':71}
    num_round = 100

    #XGBOOSTモデルに機械学習
    watchlist = [(dtrain,'train'),(dvalid,'eval')]
    model = xgb.train(params,dtrain,num_round,early_stopping_rounds=10,evals=watchlist)
    va_pred = model.predict(dvalid)
    
    #テストデータとその予測結果を表示
    print(list(va_y))
    print(va_pred)
    
    #モデルの性能を表示
    print('MAE:',mean_absolute_error(va_y,va_pred))
    print('MSE:',mean_squared_error(va_y,va_pred))
    print('RMSE:',np.sqrt(mean_squared_error(va_y,va_pred)))
    
    return model


In [ ]:
model = RERL(df)

In [ ]:
df_test.head(10)

In [ ]:
#学習データを作成する関数
#Real_Estate_Data_Preprocessing
def REDP(d_list):
    #データフレームに変換
    df = pd.DataFrame(d_list)
    
    #df内の余計な文字(\n等)を消去する
    df['礼金'] = df['礼金'].str.replace(r'\(.+\)','')
    for element in ['住所','交通','専有面積','間取り','築年数','礼金','敷金']:
        df[element] = df[element].str.translate(str.maketrans({'-':'','\r':'','\n':'','\t':'','バ':'','ス':'','分':'','徒':'','歩':'','停':'','―':'','築':'','年':'','新':'','万':'','(':'','円':''}))
    
    #ラベルエンコーディングを行い、文字列を数値化する
    #ラベルエンコーディングのモデルを初期化していたから、本来の力を発揮できていなかった
    #LE = LabelEncoder()
    df['住所'] = LE1.transform(df['住所'])
    df['路線'] = LE2.transform(df['路線'])
    df['間取り'] = LE3.transform(df['間取り'])
    
    #欠損値をどうにかする
    df.replace('',0,inplace=True)
    
    #データ型をfloat64で統一する。
    df = df.astype('float64')
    
    #入力データと正解データに分ける
    #Fact = df['賃料']
    #Input = df.drop(['賃料'],axis=1)
    
    return df


In [ ]:
df_t = REDP(d_t_list)
df_t.head(10)

In [ ]:
print('df_test:{}'.format(len(df_test['住所'].unique())))
print('df:{}'.format(len(df['住所'].unique())))

In [ ]:
#SBRLで作ったXGBOOSTモデルを用いて、df_testを入力として賃料を予測する関数を作る
#Real_Estate_Rent_Divided_Prediction
def RERDP(df):
    #説明変数と目的変数の分離
    df_fact = df['賃料']
    df_input = df.drop(['賃料'],axis=1)
    
    #予測値の生成
    df_input = xgb.DMatrix(df_input)
    pred = model.predict(df_input)
    
    #実値と予測値の表示
    #df_fact = df_fact.rename({name:'実値'})
    df_pred = pd.DataFrame(pred,columns=['予測値'])
    df_final = pd.concat([df_fact,df_pred],axis=1)
    print(df_final)
    
    #モデルの性能を表示
    print('MAE:',mean_absolute_error(df_fact,df_pred))
    print('MSE:',mean_squared_error(df_fact,df_pred))
    print('RMSE:',np.sqrt(mean_squared_error(df_fact,df_pred)))


In [ ]:
RERDP(df_t)

In [ ]:
adressC,stationC,layoutC = LE1.classes_,LE2.classes_,LE3.classes_
print(adressC,'\n','\n',stationC,'\n','\n',layoutC)

In [ ]:
#お待ちかねのオリジナル予測！
#学習した範囲内の任意のデータを入力して予測できる！
#Real_Estate_Own_Data_Prediction
def REODP(address,station,access,mc_fees,k_fees,s_fees,area,layout,age):
    d_try = {
        '住所':address,
        '路線':station,
        '交通':access,
        '管理共益費':mc_fees,
        '礼金':k_fees,
        '敷金':s_fees,
        '専有面積':area,
        '間取り':layout,
        '築年数':age
    }
    df_try = pd.DataFrame(d_try,index=['own'])
    display(df_try)

    df_try.住所 = LE1.transform(df_try.住所)
    df_try.路線 = LE2.transform(df_try.路線)
    df_try.間取り = LE3.transform(df_try.間取り)
    display(df_try)
    
    df_try = df_try.astype('float64')
    df_try = xgb.DMatrix(df_try)
    
    return print(model.predict(df_try))


In [ ]:
REODP(
    address=''
    ,station=''
    ,access=0
    ,mc_fees=0
    ,k_fees=0
    ,s_fees=0
    ,area=0
    ,layout=''
    ,age=0
)